In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

to do
- map city number to city string
- convert to full long and latiti --> need to use .apply
- combine all steps to get new test.csv

In [2]:
scrambled_df = pd.read_csv('og_data.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_false.csv')
current_df = pd.concat([train_df,test_df])

In [3]:
scrambled_df = scrambled_df.sort_values('RowId')
scrambled_df = scrambled_df.drop(columns=['Unnamed: 0','DistanceToFirstStop_p20',
       'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50','EntryHeading',
       'EntryStreetName', 'ExitHeading', 'ExitStreetName',
       'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80','TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
       'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
       'TimeFromFirstStop_p80', 'TotalTimeStopped_p20', 'TotalTimeStopped_p40',
       'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80','ValidationGroup',
        'EntryStreetMissing', 'ExitStreetMissing', 'CMWH', 'DiffHeading',
       'Rainfall', 'Temperature', 'EntryType', 'ExitType', 'Intersection',
       'SameStreet', 'LatitudeDist', 'LongitudeDist', 'CenterDistL1',
       'CenterDistL2', 'Longitude3Count', 'Latitude3Count',
       'ExitStreetNameCount', 'EntryStreetNameCount', 'IntersectionCount',
       'PathCount', 'Longitude3UniqueIntersections',
       'Latitude3UniqueIntersections', 'ExitStreetNameUniqueIntersections',
       'EntryStreetNameUniqueIntersections', 'random','IsTrain', 'Latitude', 'Longitude','Latitude3','Longitude3'
       ])

In [4]:
scrambled_df.loc[scrambled_df['City'] == 0,'City'] = 'Atlanta'
scrambled_df.loc[scrambled_df['City'] == 1,'City'] = 'Boston'
scrambled_df.loc[scrambled_df['City'] == 2,'City'] = 'Chicago'
scrambled_df.loc[scrambled_df['City'] == 3,'City'] = 'Philadelphia'

In [5]:
scrambled_df[['EntryStreetName','EntryHeading','ExitStreetName','ExitHeading']] = scrambled_df.copy()['Path'].str.split('_',expand=True)

In [6]:
scrambled_df.head()

,City,Hour,IntersectionId,Month,Path,RowId,Weekend,EntryStreetName,EntryHeading,ExitStreetName,ExitHeading
304635,Atlanta,0,1,6,Peachtree Street Southwest_SW_Mitchell Street ...,0,0,Peachtree Street Southwest,SW,Mitchell Street Southwest,SE
303418,Atlanta,0,1,6,Peachtree Street Southwest_SW_Peachtree Street...,1,0,Peachtree Street Southwest,SW,Peachtree Street Southwest,SW
303257,Atlanta,1,1,6,Peachtree Street Southwest_NE_Peachtree Street...,2,0,Peachtree Street Southwest,NE,Peachtree Street Southwest,NE
303419,Atlanta,1,1,6,Peachtree Street Southwest_SW_Peachtree Street...,3,0,Peachtree Street Southwest,SW,Peachtree Street Southwest,SW
303258,Atlanta,2,1,6,Peachtree Street Southwest_NE_Peachtree Street...,4,0,Peachtree Street Southwest,NE,Peachtree Street Southwest,NE


In [7]:
cities = ['Atlanta','Boston','Chicago','Philadelphia']
dfs = []
for city in cities:
    sub_scrambled = scrambled_df[scrambled_df['City'] == city]
    sub_current = current_df[current_df['City'] == city]
    int_id_to_ll = sub_current.groupby('IntersectionId').agg({'Longitude':'first','Latitude':'first'})
    merged_df = pd.merge(sub_scrambled, int_id_to_ll, on='IntersectionId', how='left')
    dfs.append(merged_df)

In [8]:
final_df = pd.concat(dfs)
final_df = final_df[test_df.columns]

In [9]:
final_df

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,Month,Path,City
0,0,1,33.750940,-84.393032,Peachtree Street Southwest,Mitchell Street Southwest,SW,SE,0,0,6,Peachtree Street Southwest_SW_Mitchell Street ...,Atlanta
1,1,1,33.750940,-84.393032,Peachtree Street Southwest,Peachtree Street Southwest,SW,SW,0,0,6,Peachtree Street Southwest_SW_Peachtree Street...,Atlanta
2,2,1,33.750940,-84.393032,Peachtree Street Southwest,Peachtree Street Southwest,NE,NE,1,0,6,Peachtree Street Southwest_NE_Peachtree Street...,Atlanta
3,3,1,33.750940,-84.393032,Peachtree Street Southwest,Peachtree Street Southwest,SW,SW,1,0,6,Peachtree Street Southwest_SW_Peachtree Street...,Atlanta
4,4,1,33.750940,-84.393032,Peachtree Street Southwest,Peachtree Street Southwest,NE,NE,2,0,6,Peachtree Street Southwest_NE_Peachtree Street...,Atlanta
...,...,...,...,...,...,...,...,...,...,...,...,...,...
874011,1920330,1990,39.938232,-75.149930,South 4th Street,South 4th Street,S,S,17,1,12,South 4th Street_S_South 4th Street_S,Philadelphia
874012,1920331,1990,39.938232,-75.149930,South 4th Street,South 4th Street,S,S,18,1,12,South 4th Street_S_South 4th Street_S,Philadelphia
874013,1920332,1990,39.938232,-75.149930,South 4th Street,Catharine Street,S,W,21,1,12,South 4th Street_S_Catharine Street_W,Philadelphia
874014,1920333,1990,39.938232,-75.149930,South 4th Street,South 4th Street,S,S,21,1,12,South 4th Street_S_South 4th Street_S,Philadelphia


In [10]:
final_df.to_csv('test.csv',index=False)